# Homework: Gaussian Quadrature Integration

###### GH: Gauss-Hermite; GL: Gauss-Legendre; GLa: Gauss-Laguerre

For all the problems, you may obtain $\{\omega_i, \xi_i \}$ directly from Julia's `FastGaussQuadrature` packages. For instance, `xi, wi = gausslegendre(20)` would give $\{\omega_i, \xi_i \}$, $i=1,\ldots,20$, from the Gauss-Legendre rule.

In [13]:
#Pkg.add("QuadGK")
#Pkg.add("FastGaussQuadrature")

   Resolving package versions...
   Installed FastGaussQuadrature ─ v0.5.0
    Updating `C:\Users\d3m2w\.julia\environments\v1.8\Project.toml`
  [442a2c76] + FastGaussQuadrature v0.5.0
    Updating `C:\Users\d3m2w\.julia\environments\v1.8\Manifest.toml`
  [442a2c76] + FastGaussQuadrature v0.5.0
Precompiling project...
  ✓ FastGaussQuadrature
  1 dependency successfully precompiled in 3 seconds. 222 already precompiled.


## This problem asks you to adopt the Gauss-Hermite rule to an infinite integral where the weight function does not match.  The integration problem is:
  $$\begin{aligned}
   I = \int_{-\infty}^\infty g(x) d x =  \int_{-\infty}^\infty \exp\left(-\frac{1}{3}x^2\right)\sqrt{1+x^2} dx.
  \end{aligned}$$ 
 

- ##### Let's do this without using changes of variables. 
  - Transform $g(x)$ to $e^{-x^2}f(x)$ so that the Gauss-Hermite rule could be applied. Write down the re-formulated integration problem explicitly and write down $f(x)$ explicitly.
  - Write Julia code to integrate the function numerically.

 $$\begin{aligned}
   I =  \int_{-\infty}^\infty \exp(-x^2)\exp\left(\frac{2}{3}x^2\right)\sqrt{1+x^2} dx.
  \end{aligned}$$ 

In [2]:
#1.1
using QuadGK
answer = quadgk(x -> exp(-(x^2)/3)sqrt(1+x^2), -Inf, Inf)
print(answer[1])

4.563813002879844

## Write a Julia function with the following specifications.
  - It calculates $E[g(y)]$ for any arbitrary $g(y)$ where $y \sim N(\mu, \sigma^2)$.
  - User's inputs include $g(y)$, $\mu$, $\sigma^2$, and $n$.
  - You may obtain $\{\omega_i, \xi_i\}$ directly from Julia's package.
  - If $g(y) = 1 + 3y + y^2$, $y \sim N(1,2)$, and $n=30$, the interface could be like `myInteg(1 + 3y + y^2, mean=1, variance=2, n=30)`, or, `myInteg(g, mean=1, variance=2, n=30)`.
  - Hint: You may define the probability density function of $y$ using the `Distributions` package and then `pdf(Normal(mean, std.dev.),x)`.

In [20]:
using Distributions
using FastGaussQuadrature
function Intg(g, mean, vari, n)
    xi, wi = gausshermite(n)
    answer = sum( wi .* g.(sqrt(2*vari)*xi .+ mean))/sqrt(pi)
end

#test
g(y) = 1+3*y+y^2
print(Intg(g , 1, 2, 30))

6.999999999999993

## It is generally true that proper integrals (i.e., with finite domains) are numerically easier to deal with. But what if a problem is naturally a GH type with an infinite domain and we transform it to a GL problem with a finite domain? Is the GL rule better in this scenario? Let's find out the answer. Consider the following integration problem which is often encountered in econometric analysis. 

$$\begin{aligned}
      I = \int_{-\infty}^\infty \Phi\left(\frac{x-a}{b}\right)\phi(x) dx,
\end{aligned}$$

where $X\sim N(0,1)$ is a standard normal random variable, $\phi(z)$ is the density function (PDF) of a random variable $z$, and $\Phi(z)$ is the CDF of $z$.
   - ##### Write a Julia function that uses the GH rule to approximate the integral. The program should be a function of $a$ and $b$.
   - ##### Transform this problem to the GL type. What is the transformation rule and what is the Jacobian? Type the equations and show your work.
   - ##### Write a Julia function that uses the GL rule to approximate the integral. The program should be a function of $a$ and $b$.
   - ##### Suppose the required precision is 6 digits after the decimal. Compare the performances of these functions. Which one converges more quickly (smaller $n$)? You may pick values of $a$ and $b$ to do the comparison. You may draw graphs to compare the results.
   - ##### Based on your results, is the GL rule always the most efficient?

Transform I to GL type.
$$\begin{aligned}
      I = \int_{-1}^1 \Phi\left(\frac{\frac{t}{1-t^2}-a}{b}\right)\phi(\frac{t}{1-t^2})\frac{t^2 + 1}{(t^2 -1)^2} dt,
\end{aligned}$$

In [73]:
using Distributions
using FastGaussQuadrature

function IntgGH(a, b, n)
    xi, wi = gausshermite(n)
    g(x) = cdf(Normal(-a/b, 1/b), x) * pdf(Normal(0, 1), x) 
    x(t) = sqrt(2)*t
    f(t) = g(x(t))*sqrt(2)/exp(-t^2)
    answer = sum( wi .* f.(xi))
end

function IntgGL(a, b, n)
    xi, wi = gausslegendre(n)
    g(x) = cdf(Normal(-a/b, 1/b), x) * pdf(Normal(0, 1), x) 
    f(t) = g(t/(1-t^2))*(t^2+1)/((t^2-1)^2)
    answer = sum( wi .* f.(xi))
end

prev = 0
n = 0
while true
    n += 1
    if round(prev; digits = 6) == round(IntgGH(1, 2 , n); digits = 6)
        print("GH method needs n >= ",n," to converge.\n")
        break
    else
        prev = IntgGH(1, 2 , n)
    end
end

prev = 0
n = 0
while true
    n += 1
    if round(prev; digits = 6) == round(IntgGL(1, 2 , n); digits = 6)
        print("GL method needs n >= ",n," to converge.\n")
        break
    else
        prev = IntgGL(1, 2 , n)
    end
end
print("GL rule not always the most efficient.")

GH method needs n >= 35 to converge.
GL method needs n >= 40 to converge.
GL rule not always the most efficient.

## This problem asks you to consider an integration problem where it can be transformed to different types to use different Gaussian quadrature rules. Which one is the best? The integration problem is:

 $$\begin{aligned}
 \int_{a}^\infty \exp\left(-\frac{1}{100}x^2\right)(1+x^2) dx,
 \end{aligned}$$

where $a$ is finite but not necessarily 0. This problem has some characteristics of the GLa and GH types: It has an exponential function, and by mapping $a$ to $-1$, $0$ or $-\infty$ via change of variables, the domain would be of the GL, GLa and GH types, respectively. You are asked to perform the approximation using rules of each of the types and compare the performance. In the code, make $a$ a parameter that we can assign a value to it.

 - ##### Write Julia code that uses the GL rule to approximate the integral. 
 - ##### Write Julia code that uses the GLa rule to approximate the integral. 
 - ##### Write Julia code that uses the GH rule to approximate the integral. 
 - ##### Suppose the required precision is 6 digits after the decimal. Compare the performances of these functions; drawing a graph will be useful. You may pick a value for $a$ in the comparison. 
 - ##### According to your result above, which rule converges more quickly (smaller $n$)? What is the intuition behind the result?

In [20]:
using Distributions
using FastGaussQuadrature
function IntgGLa(a, n)
    xi, wi = gausslaguerre(n)
    g(t) = exp(-(t-a)^2/100)*(1+(t-a)^2)
    f(t) = g(t)/exp(-(t-a))
    answer = sum( wi .* f.(xi))
end

function IntgGL(a, n)
    xi, wi = gausslegendre(n)
    g(t) = exp(-((a + (1+t)/(1-t))^2/100))*(1+(a + (1+t)/(1-t))^2)
    f(t) = g(t)*(2/((t-1)^2))
    answer = sum( wi .* f.(xi))
end

prev = 0
n = 0

while true
    n += 1
    if round(prev; digits = 6) == round(IntgGLa(1, n); digits = 6)
        print("GLa method needs n >= ",n," to converge.\n")
        break
    else
        prev = IntgGLa(1, n)
    end
end

prev = 0
n = 0
while true
    n += 1
    if round(prev; digits = 6) == round(IntgGL(1, n); digits = 6)
        print("GL method needs n >= ",n," to converge.\n")
        break
    else
        prev = IntgGL(1, n)
    end
end
print("I dunno how to map the domain to fit GH, so I skip that.\n")
print("GLa converges more quickly, because original domain is similar to GLa?")

GLa method needs n >= 15 to converge.
GL method needs n >= 93 to converge.
I dunno how to map the domain to fit GH, so I skip that.
GLa converges more quickly, because original domain is similar to GLa?